# Kuantisasi Pytorch

PyTorch mendukung kuantisasi INT8 dibandingkan dengan model FP32 pada umumnya yang memungkinkan pengurangan ukuran model sebesar 4x dan pengurangan kebutuhan bandwidth memori sebesar 4x
sambil tetap mencapai akurasi yang sebanding untuk banyak aplikasi. Buku catatan ini menunjukkan cara mengkuantisasi model dari FP32 ke INT8 menggunakan alat kuantisasi PyTorch. Kami akan melatih model CNN sederhana di mnist dan kemudian mengkuantisasinya menggunakan alat kuantisasi dan membandingkan akurasi dan ukuran model terkuantisasi dengan model FP32 asli.

## Siapkan PyTorch

Pertama, mari instal PyTorch dan torchvision lalu impor modul yang diperlukan.

In [1]:
%pip install torch torchvision

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torch.quantization
import pathlib

## Kuantisasi Dinamis

Untuk kuantisasi dinamis, bobot dikuantisasi tetapi aktivasi dibaca atau disimpan dalam floating point dan aktivasi hanya dikuantisasi untuk komputasi.

### Muat kumpulan data MNIST

Pertama, kita memuat dataset MNIST

In [3]:
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])

train_dataset = datasets.MNIST('./data', train=True, download=True,transform=transform)
test_dataset = datasets.MNIST('./data', train=False,transform=transform)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:00<00:00, 15.2MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 539kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:00<00:00, 4.51MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 7.08MB/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



### Latih Modelnya

Selanjutnya, kita mendefinisikan model CNN sederhana dan kemudian melatih dataset MNIST

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=12, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc = nn.Linear(12 * 13 * 13, 10)

    def forward(self, x):
        x = x.reshape(-1, 1, 28, 28)
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = x.reshape(x.size(0), -1)
        x = self.fc(x)
        output = F.log_softmax(x, dim=1)
        return output


train_loader = torch.utils.data.DataLoader(train_dataset, 32)
test_loader = torch.utils.data.DataLoader(test_dataset, 32)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

epochs = 1

model = Net().to(device)
optimizer = optim.Adam(model.parameters())

model.train()

for epoch in range(1, epochs+1):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()))

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.411571
Train Epoch: 1 [32/60000 (0%)]	Loss: 2.222979
Train Epoch: 1 [64/60000 (0%)]	Loss: 1.987400
Train Epoch: 1 [96/60000 (0%)]	Loss: 1.835723
Train Epoch: 1 [128/60000 (0%)]	Loss: 1.976682
Train Epoch: 1 [160/60000 (0%)]	Loss: 2.187811
Train Epoch: 1 [192/60000 (0%)]	Loss: 1.348413
Train Epoch: 1 [224/60000 (0%)]	Loss: 1.647015
Train Epoch: 1 [256/60000 (0%)]	Loss: 1.649863
Train Epoch: 1 [288/60000 (0%)]	Loss: 1.196274
Train Epoch: 1 [320/60000 (1%)]	Loss: 1.283170
Train Epoch: 1 [352/60000 (1%)]	Loss: 1.180266
Train Epoch: 1 [384/60000 (1%)]	Loss: 1.152334
Train Epoch: 1 [416/60000 (1%)]	Loss: 1.101845
Train Epoch: 1 [448/60000 (1%)]	Loss: 0.994502
Train Epoch: 1 [480/60000 (1%)]	Loss: 1.439544
Train Epoch: 1 [512/60000 (1%)]	Loss: 1.142379
Train Epoch: 1 [544/60000 (1%)]	Loss: 0.850681
Train Epoch: 1 [576/60000 (1%)]	Loss: 1.111073
Train Epoch: 1 [608/60000 (1%)]	Loss: 1.185717
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.071345
Train Epoch: 1 [67

### Model Kuantisasi

Setelah pelatihan, kita dapat mengkuantisasi model menggunakan fungsi `torch.quantization.quantize_dynamic` dari pytorch.

In [5]:
model.to('cpu')
quantized_model = torch.quantization.quantize_dynamic(model, {torch.nn.Linear}, dtype=torch.qint8)

### Periksa Ukuran Model

Kita dapat melihat bahwa model terkuantisasi jauh lebih kecil dari model aslinya

In [6]:
models_dir = pathlib.Path("./models/")
models_dir.mkdir(exist_ok=True, parents=True)
torch.save(model.state_dict(), "./models/original_model.p")
torch.save(quantized_model.state_dict(), "./models/quantized_model.p")

%ls -lh models

total 108K
-rw-r--r-- 1 root root 82K Jan  3 15:50 original_model.p
-rw-r--r-- 1 root root 23K Jan  3 15:50 quantized_model.p


### Periksa Akurasi

Kita dapat melihat bahwa model terkuantisasi memiliki akurasi yang sebanding dengan model aslinya

In [7]:
def test(model, device, data_loader, quantized=False):
    model.to(device)
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(data_loader.dataset)

    return 100. * correct / len(data_loader.dataset)

original_acc = test(model, "cpu", test_loader)
quantized_acc = test(quantized_model, "cpu", test_loader)

print('Original model accuracy: {:.0f}%'.format(original_acc))
print('Quantized model accuracy: {:.0f}%'.format(quantized_acc))

Original model accuracy: 97%
Quantized model accuracy: 96%


## Kuantisasi Statis Pasca Pelatihan

Kuantisasi statis pasca pelatihan adalah saat bobot dan aktivasi dikuantisasi dan kalibrasi diperlukan pasca pelatihan. Di sini kami mengkuantisasi model menggunakan fungsi `torch.quantization.quantize_fx()` dari PyTorch dan membandingkan akurasi dan ukuran model terkuantisasi dengan model FP32 asli.

Untuk melakukan kuantisasi menggunakan alat kuantisasi statis pasca-pelatihan, pertama-tama tentukan model atau muat model yang telah dilatih sebelumnya, lalu buat pemetaan konfigurasi kuantisasi menggunakan default untuk mesin QNNPACK. Atur model ke mode evaluasi dan buat tensor input sampel. Kemudian, siapkan model untuk kuantisasi menggunakan fungsi `quantize_fx.prepare_fx()`. Hal ini melibatkan penerapan pemetaan konfigurasi kuantisasi dan menyiapkan model untuk menangani presisi int8. Model yang telah disiapkan kemudian dieksekusi pada tensor masukan. Terakhir, model terkuantisasi dengan memanggil`quantize_fx.convert_fx()` dan menyimpan model ke disk.

In [8]:
from torch.ao.quantization import (
  get_default_qconfig_mapping,
  get_default_qat_qconfig_mapping,
  QConfigMapping,
)
import torch.ao.quantization.quantize_fx as quantize_fx
import copy

loaded_model = Net()
loaded_model.load_state_dict(torch.load("./models/original_model.p"))
model_to_quantize = copy.deepcopy(loaded_model)

qconfig_mapping = get_default_qconfig_mapping("qnnpack")
model_to_quantize.eval()

input_fp32 = next(iter(test_loader))[0][0:1]
input_fp32.to('cpu')

model_fp32_prepared = quantize_fx.prepare_fx(model_to_quantize, qconfig_mapping, input_fp32)
model_fp32_prepared(input_fp32)
model_int8 = quantize_fx.convert_fx(model_fp32_prepared)

torch.save(model_int8.state_dict(), "./models/post_quantized_model.p")

<ipython-input-8-7c51cb5ca612>:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model.load_state_dict(torch.load("./models/original_model.p"))


## Periksa Ukuran Model

Sekali lagi, kita dapat melihat bahwa model terkuantisasi jauh lebih kecil dibandingkan model aslinya## Periksa Ukuran Model

Sekali lagi, kita dapat melihat bahwa model terkuantisasi jauh lebih kecil dibandingkan model aslinya

In [9]:
%ls -lh models

total 136K
-rw-r--r-- 1 root root 82K Jan  3 15:50 original_model.p
-rw-r--r-- 1 root root 25K Jan  3 15:50 post_quantized_model.p
-rw-r--r-- 1 root root 23K Jan  3 15:50 quantized_model.p


## Periksa Akurasi

Sekali lagi, kita dapat melihat bahwa akurasi model terkuantisasi tidak jauh berbeda dengan akurasi aslinya

In [10]:
quantized_acc = test(model_int8, "cpu", test_loader, quantized=True)
print('Post quantized model accuracy: {:.0f}%'.format(quantized_acc))

Post quantized model accuracy: 96%
